In [1]:
import sys
sys.path.append("../")
import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split

from adabelief_pytorch import AdaBelief

from wrapper import Variational_Flipout, Variational_LRT

In [2]:
data = load_boston()

def standarize(x):
    return (x - x.mean(0, keepdims = True)) / x.std(0, keepdims = True)

X = data['data']
Y = data['target']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state=5)
X_train = standarize(torch.from_numpy(X_train)).to(torch.float32)
Y_train = standarize(torch.from_numpy(Y_train)).to(torch.float32)
X_test = standarize(torch.from_numpy(X_test)).to(torch.float32)
Y_test = standarize(torch.from_numpy(Y_test)).to(torch.float32)

train_set = TensorDataset(X_train, Y_train)
test_set = TensorDataset(X_test, Y_test)
train_loader = DataLoader(train_set, batch_size=16, shuffle=True)
test_loader = DataLoader(test_set, batch_size=16, shuffle=False)

/home/nonconvexopt/.local/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this case special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California h

In [3]:
class net_Flipout(nn.Module):
    def __init__(self):
        super(net_Flipout, self).__init__()
        self.l1 = Variational_Flipout(nn.Linear(13, 50))
        self.l2 = Variational_Flipout(nn.Linear(50, 1))
        
    def forward(self, x):
        x = self.l1(x)
        x = F.silu(x)
        x = self.l2(x)
        return x
    
    def kld(self):
        sum_kl = 0.0
        for module in self.modules():
            if isinstance(module, Variational_Flipout):
                sum_kl += module.kld()
        return sum_kl

In [197]:
model = net_Flipout().cuda()
num_epochs = 200
lr = 1e-2
criterion = nn.MSELoss()
optimizer = AdaBelief(
    model.parameters(),
    lr=lr, betas=(0.9, 0.999), eps=1e-8, weight_decouple = True, rectify=False, 
)

Please check your arguments if you have upgraded adabelief-pytorch from version 0.0.5.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  ---------
adabelief-pytorch=0.0.5  1e-08  False              False
>=0.1.0 (Current 0.2.0)  1e-16  True               True
SGD better than Adam (e.g. CNN for Image Classification)    Adam better than SGD (e.g. Transformer, GAN)
----------------------------------------------------------  ----------------------------------------------
Recommended eps = 1e-8                                      Recommended eps = 1e-16
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer
You can disable the log message by setting "print_change_log = False", though it is recommended to keep as a reminder.

Weight decoupling enabled in AdaBelief


In [4]:
class net_LRT(nn.Module):
    def __init__(self):
        super(net_LRT, self).__init__()
        self.l1 = Variational_LRT(nn.Linear(13, 50))
        self.l2 = Variational_LRT(nn.Linear(50, 1))
        
    def forward(self, x):
        x = self.l1(x)
        x = F.silu(x)
        x = self.l2(x)
        return x
    
    def kld(self):
        sum_kl = 0.0
        for module in self.modules():
            if isinstance(module, Variational_LRT):
                sum_kl += module.kld()
        return sum_kl

In [198]:
for epoch in range(num_epochs):
    running_loss = 0.0
    total_loss = 0.0
    for ind, (x, y) in enumerate(train_loader):
        optimizer.zero_grad()
        loss = criterion(model(x.cuda()).squeeze(), y.cuda())
        kld = model.kld()
        (loss + kld).backward()
        optimizer.step()
        running_loss += loss.item()
        total_loss += loss.item()
        if ind % 50 == 1:    # print every 100 mini-batches
            print('[%d, %5d] loss: %.6f, kld: %.6f' %
                  (epoch + 1, ind + 1, running_loss / 50, kld))
            running_loss = 0.0

[1,     2] loss: 0.044341, kld: 7.966604
[2,     2] loss: 0.026021, kld: 0.456928
[3,     2] loss: 0.024317, kld: 0.283378
[4,     2] loss: 0.018345, kld: 0.317281
[5,     2] loss: 0.033388, kld: 0.341864
[6,     2] loss: 0.012024, kld: 0.341758
[7,     2] loss: 0.012887, kld: 0.303652
[8,     2] loss: 0.030146, kld: 0.311387
[9,     2] loss: 0.023699, kld: 0.279556
[10,     2] loss: 0.031999, kld: 0.298626
[11,     2] loss: 0.018676, kld: 0.323895
[12,     2] loss: 0.024932, kld: 0.345561
[13,     2] loss: 0.042010, kld: 0.308149
[14,     2] loss: 0.031004, kld: 0.339616
[15,     2] loss: 0.023130, kld: 0.219635
[16,     2] loss: 0.023056, kld: 0.330282
[17,     2] loss: 0.009161, kld: 0.330651
[18,     2] loss: 0.013812, kld: 0.270136
[19,     2] loss: 0.025652, kld: 0.334141
[20,     2] loss: 0.029415, kld: 0.347452
[21,     2] loss: 0.028381, kld: 0.296811
[22,     2] loss: 0.024603, kld: 0.262316
[23,     2] loss: 0.034323, kld: 0.335438
[24,     2] loss: 0.029761, kld: 0.357860
[

In [ ]:
for epoch in range(num_epochs):
    running_loss = 0.0
    total_loss = 0.0
    for i, data in enumerate(train_loader):
        optimizer.zero_grad()
        model_state_dict = copy.deepcopy(net_fn.state_dict())
        #loss = - log_posterior_fn(model_state_dict, data)
        loss = - log_posterior_fn(net_fn, model_state_dict, data)
        loss.backward()
        optimizer.step()
        scheduler1.step()
        running_loss += loss.item()
        total_loss += loss.item()
        if i % 100 == 99:    # print every 100 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0
    model_state_dict = copy.deepcopy(net_fn.state_dict())
    test_acc, all_test_probs = evaluate_fn(test_loader, model_state_dict)
    scheduler2.step(test_acc)
    
    print("Epoch {}".format(epoch))
    print("\tAverage loss: {}".format(total_loss / epoch_steps))
    print("\tTest accuracy: {}".format(test_acc))